In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import hit_rates as hr
import rankfmlib as fmlib
# new library for performance studies built from rankfmlib to read a single file rather than yearly files.
import newlib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from rankfm.rankfm import RankFM
import function_lib as flib

from rankfm.evaluation import hit_rate

Read member attributes

In [5]:
# Perhaps add an argument stating which colums are attributes
interaction_dct = newlib.read_data_attributes_single_file("activity_reduced_with_attributes.csv",
                                                        age_cuts=[0,30,50,70,150])
# df_member_attrib #[['avg_yr_l','avg_yr_h']]

Index(['MEMBER_ID', 'TRUE_OD', 'D', 'FLIGHT_DATE', 'BOOKING_DATE',
       'TICKET_SALES_DATE', 'TICKET_NUMBER', 'TRUE_ORIGIN_COUNTRY',
       'ADDR_COUNTRY', 'PNR', 'PARTY_SZ', 'size', 'booking_date',
       'booking_dowk', 'booking_mo', 'flight_date', 'flight_dowk', 'flight_mo',
       'TIER_LEVEL', 'GENDER', 'BIRTH_DATE', 'NATIONALITY', 'ADDR_COUNTRY_y',
       'age_at_flight'],
      dtype='object') (218298, 24)


In [6]:
interaction_dct.keys()
newlib.train_valid_dct(interaction_dct, train_perc=0.7, valid_perc=0.1)
print(interaction_dct.keys())

dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])


In [7]:
interaction_dct.keys(), interaction_dct['df_user_attr'];

In [8]:
data_train = interaction_dct['data_train']
data_valid = interaction_dct['data_valid']
df_members = interaction_dct['df_members']
data_train.shape, df_members.shape, 

((152810, 2), (218298, 2))

In [9]:
data_train.MEMBER_ID.nunique()

41745

In [10]:
df_user_attrib = interaction_dct['df_user_attr']
data_train.MEMBER_ID.nunique(), df_user_attrib.MEMBER_ID.nunique()

(41745, 46072)

In [11]:
# Check that member_attrib is a superset of data_train
df_member_attrib = interaction_dct['df_user_attr']
set(data_train.MEMBER_ID) - set(df_member_attrib.MEMBER_ID)

set()

In [12]:
data_train.shape, data_valid.shape

((152810, 2), (21829, 2))

In [13]:
mtrain = set(data_train.MEMBER_ID)
mvalid = set(data_valid.MEMBER_ID)
len(mtrain), len(mvalid), # mvalid.intersection(mtrain), #mtrain

(41745, 15752)

In [14]:
interaction_dct.keys()

dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])

In [15]:
interaction_dct['df_user_attr'].head(2)

,MEMBER_ID,"age_dep_(0, 30]","age_dep_(30, 50]","age_dep_(50, 70]","age_dep_(70, 150]"
0,100031203,0,0,1,0
3,230019464,0,0,1,0


In [16]:
loss = 'warp'
loss = 'bpr'  # nb neg samples = 1
model = RankFM(factors=20, loss=loss, max_samples=10000, alpha=0.05, beta=0.1, learning_rate=0.1, learning_schedule='constant')

The problem to solve: Why are results worse with filter=False? To figure this out, I may have to write my own recommender and hit_rate functions. 
 1) optimize parameter without attributes
 2) optimize age brackets using just one attribute
 3) optimize temperature brackets (destination features) using just one attribute
 4) add gender, country of origin to the member attributes

In [20]:
newlib.run_model(model, interaction_dct, nb_epochs=120)

dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.29701704545454544
hr (previous not filtered):  0.2122159090909091


___

In [21]:
nb_epochs = [50, 100, 250, 500, 1000]
for epochs in nb_epochs:
    print("-- nb epochs: ", epochs)
    newlib.run_model(model, interaction_dct, nb_epochs=epochs)
    print("------------------------------------------")

-- nb epochs:  [50, 100, 250, 500, 1000]
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.26704545454545453
hr (previous not filtered):  0.2046875
------------------------------------------
-- nb epochs:  [50, 100, 250, 500, 1000]
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.31150568181818183
hr (previous not filtered):  0.2369318181818182
------------------------------------------
-- nb epochs:  [50, 100, 250, 500, 1000]
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.3221590909090909
hr (previous not filtered):  0.22876420454545454
------------------------------------------
-- nb epochs:  [50, 100, 250, 500, 1000]
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.32911931818181

In [22]:
nb_epochs = [50, 100, 250, 500, 1000]
for epochs in nb_epochs:
    print("-- nb epochs: ", epochs)
    newlib.run_model(model, interaction_dct, nb_epochs=epochs)
    print("------------------------------------------")

-- nb epochs:  50
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.2720880681818182
hr (previous not filtered):  0.20717329545454546
------------------------------------------
-- nb epochs:  100
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.30170454545454545
hr (previous not filtered):  0.23423295454545454
------------------------------------------
-- nb epochs:  250
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.31079545454545454
hr (previous not filtered):  0.2317471590909091
------------------------------------------
-- nb epochs:  500
dict_keys(['df_members', 'df_user_attr', 'df_item_attr', 'data_train', 'data_valid', 'data_test'])
hr (previous filtered):  0.32457386363636365
hr (previous not filtered):  0.22450284090909092
--------------------------

In [ ]:
members_train = data_train.MEMBER_ID.unique()  # returns ndarray
members_valid = data_valid.MEMBER_ID.unique()

In [ ]:
mtrain = set(members_train)
mvalid = set(members_valid)
len(mvalid), len(mtrain), mvalid.intersection(mtrain);

In [ ]:
model.recommend(members_valid, n_items=5, filter_previous=True, cold_start='drop')

In [ ]:
members_valid.shape

In [ ]:
mvalid_np = np.asarray(mvalid)

In [ ]:
data_valid

In [ ]:
data_test

In [ ]:
hit_rate(model, data_test, k=5, filter_previous=True)

In [ ]:
# More iterations, worse results. Why would that be? Unless iterations is not converged?
interactions_dct, model = newlib.calculate_dct_with_attributes(nb_samples=1000, with_attrib=True, verbose=False, nb_epochs=100, loss='warp')

In [ ]:
# Hit rate goes fro 0.29 to 0.21 when nb_epochs goes from 30 to 100. I DO NOT LIKE THIS. 
newlib.topn_recommendations_with_attributes(model, interactions_dct, topN=3)

In [ ]:
Construct training/testing datasets
# Consider a training set of 10% of the data. 
# Shuffle the data first


In [ ]:
data = interactions_dct['train']
data1 = data.sample(frac=1, random_state=100)

In [ ]:
data.head(), data1.head()

In [ ]:
data.groupby('MEMBER_ID').agg({'D':list}).shape

In [ ]:
print(x[])

In [ ]:
hit_rate(model, data, k=5, filter_previous=False) # cold_start='drop');

Calculate the hit rate myself based on the recommendations, which are based on the ranking scores. 
To do this, 
* first create a list of destinations for each member in the training set. 
* next, for each of the recommendations, determine whether the member flew there in the test set. 
* if the member flew there in the test set, add 1 to the hit list, else 0

In [ ]:
data = interactions_dct['train']
x = list(interactions_dct['train']['MEMBER_ID'])
data_train, data_valid, data_test = newlib.train_valid(x, 0.8, 0.1)

Train the model

In [ ]:
in_file = "activity_reduced_with_attributes.csv"
member_dest_df, df_user_attrib, df_item_attrib = newlib.read_data_attributes_single_file(in_file)